# 參數導入

In [9]:
import math
import numpy as np
import pandas as pd
import os
import csv

from tqdm import tqdm

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchviz import make_dot

from torch.utils.tensorboard import SummaryWriter

In [10]:
def same_seed(seed): 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    print(f'Set Seed = {seed}')


def train_valid_split(data_set, valid_ratio, seed):
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)


def predict(test_loader, model, device):
    model.eval()
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():
            pred = model(x)         
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds


# 數據集

In [11]:
class COVID19Dataset(Dataset):
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# 模型

In [12]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) 
        return x


# 特徵選取

In [13]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2,3,4] 

    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid


# 訓練

In [14]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') 

    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9) 
    
    writer = SummaryWriter()

    if not os.path.isdir('./models'):
        os.mkdir('./models')

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() 
        loss_record = []

        train_pbar = tqdm(train_loader, position=0, leave=True)
        train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
        for x, y in train_pbar:
            optimizer.zero_grad()        
            x, y = x.to(device), y.to(device) 
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                    
            optimizer.step()
            step += 1
            loss_record.append(loss.detach().item())
            
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() 
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) 
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return


# 超參數

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 20240320,   
    'select_all': True,
    'valid_ratio': 0.2,
    'n_epochs': 3000,
    'batch_size': 256, 
    'learning_rate': 1e-5,              
    'early_stop': 400,    
    'save_path': './models/model.ckpt'  
}


In [16]:
same_seed(config['seed'])

pd.set_option('display.max_column', 200)
train_df, test_df = pd.read_csv('./covid.train.csv'), pd.read_csv('./covid.test.csv')
display(train_df.head(3))
train_data, test_data = train_df.values, test_df.values
del train_df, test_df 
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

Set Seed = 20240320


,id,AL,AK,AZ,AR,CA,CO,CT,FL,GA,ID,IL,IN,IA,KS,KY,LA,MD,MA,MI,MN,MS,MO,NE,NV,NJ,NM,NY,NC,OH,OK,OR,RI,SC,TX,UT,VA,WA,cli,ili,hh_cmnty_cli,nohh_cmnty_cli,wearing_mask,travel_outside_state,work_outside_home,shop,restaurant,spent_time,large_event,public_transit,anxious,depressed,worried_finances,tested_positive,cli.1,ili.1,hh_cmnty_cli.1,nohh_cmnty_cli.1,wearing_mask.1,travel_outside_state.1,work_outside_home.1,shop.1,restaurant.1,spent_time.1,large_event.1,public_transit.1,anxious.1,depressed.1,worried_finances.1,tested_positive.1,cli.2,ili.2,hh_cmnty_cli.2,nohh_cmnty_cli.2,wearing_mask.2,travel_outside_state.2,work_outside_home.2,shop.2,restaurant.2,spent_time.2,large_event.2,public_transit.2,anxious.2,depressed.2,worried_finances.2,tested_positive.2,cli.3,ili.3,hh_cmnty_cli.3,nohh_cmnty_cli.3,wearing_mask.3,travel_outside_state.3,work_outside_home.3,shop.3,restaurant.3,spent_time.3,large_event.3,public_transit.3,anxious.3,depressed.3,worried_finances.3,tested_positive.3,cli.4,ili.4,hh_cmnty_cli.4,nohh_cmnty_cli.4,wearing_mask.4,travel_outside_state.4,work_outside_home.4,shop.4,restaurant.4,spent_time.4,large_event.4,public_transit.4,anxious.4,depressed.4,worried_finances.4,tested_positive.4
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.658466,0.724606,12.939284,8.556230,76.521693,7.863621,33.069066,64.129378,33.438714,40.539878,17.446499,3.993190,10.122837,7.841023,37.992365,7.374846,0.653157,0.713249,12.488933,8.219380,75.179558,8.083434,32.564654,65.212903,34.285572,40.697210,17.754941,3.997443,10.161621,8.067275,37.882278,7.219989,0.677783,0.737965,12.487637,8.135510,74.189695,8.001016,32.145331,66.258549,34.931095,40.889258,18.038068,3.970457,10.043014,7.983358,37.705024,7.077938,0.666751,0.723506,12.367718,8.006131,73.211902,7.873342,31.330236,66.753902,35.586606,40.741650,17.800711,3.984791,10.148898,8.262288,37.384963,7.452243,0.685628,0.740943,12.364307,8.151578,71.820231,7.944531,31.113209,67.394551,36.674291,40.743132,17.842221,4.093712,10.440071,8.627117,37.329512,7.456154
1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.693287,0.675962,15.008467,11.091818,82.375469,11.082587,36.373964,60.471825,30.516978,43.162123,16.038761,4.519762,12.133287,10.155721,33.977549,9.850027,0.738029,0.720511,15.070049,10.990937,80.943299,10.596788,36.435828,61.653141,31.449290,43.751356,16.752518,4.385694,11.911724,9.798448,32.786802,10.050547,0.831586,0.827523,14.568504,10.578924,79.096391,10.418171,35.851804,62.835720,32.802637,43.753525,17.195922,4.671201,11.418314,10.600311,32.679030,10.388084,0.767643,0.763580,14.264862,10.212529,76.645984,10.614731,34.639961,63.652437,34.147714,44.682727,17.705140,4.938357,11.056725,10.889148,32.933142,8.707858,0.713255,0.719378,12.894363,8.919288,74.687112,10.523814,33.920257,64.398380,34.612238,44.035688,17.808103,4.924935,10.172662,9.954333,32.508881,8.010957
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.617041,0.617041,8.614719,5.238097,87.040999,15.569851,33.366362,58.819368,24.674416,36.678937,13.490946,6.639301,9.753290,7.691015,38.685771,3.897851,0.663597,0.635459,8.472831,5.154184,86.743467,15.915242,33.012910,60.157822,25.322821,36.456240,13.385945,6.700146,9.949982,7.833514,38.036480,3.083111,0.534394,0.497435,8.493353,5.125553,86.647289,15.963496,32.471093,60.910411,25.158967,35.968207,13.656070,6.748557,10.664876,7.975821,37.826924,3.132834,0.536224,0.498305,8.227657,5.298912,86.285798,15.788676,31.931642,61.233289,25.156025,36.032687,13.863302,6.879646,10.731187,7.847829,37.128714,3.444182,0.479111,0.432445,8.067909,5.333533,86.312419,16.477784,31.604604,62.101064,26.521875,36.746453,13.903667,7.313833,10.388712,7.956139,36.745588,2.906977


train_data size: (2160, 118) 
valid_data size: (539, 118) 
test_data size: (1078, 117)
number of features: 117


# 模型訓練

In [17]:
model = My_Model(input_dim=x_train.shape[1]).to(device) 
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/3000]: 100%|█████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 69.44it/s, loss=55.9]


Epoch [1/3000]: Train loss: 55.4856, Valid loss: 53.5397
Saving model with loss 53.540...


Epoch [2/3000]: 100%|████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 350.91it/s, loss=40.1]


Epoch [2/3000]: Train loss: 48.2910, Valid loss: 51.3791
Saving model with loss 51.379...


Epoch [3/3000]: 100%|████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 324.97it/s, loss=44.4]


...

Epoch [2626/3000]: Train loss: 1.8083, Valid loss: 1.9813


Epoch [2627/3000]: 100%|█████████████████████████████████████████████████████| 9/9 [00:00<00:00, 351.58it/s, loss=1.93]


Epoch [2627/3000]: Train loss: 1.6283, Valid loss: 1.9222


Epoch [2628/3000]: 100%|█████████████████████████████████████████████████████| 9/9 [00:00<00:00, 359.52it/s, loss=1.65]

Epoch [2628/3000]: Train loss: 1.7147, Valid loss: 1.3781

Model is not improving, so we halt the training session.


# 訓練過程及指標可視化

In [18]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/ --port=6008

Reusing TensorBoard on port 6008 (pid 22588), started 1 day, 0:57:53 ago. (Use '!kill 22588' to kill it.)

# Testing

In [19]:
def save_pred(preds, file):
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')         

100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1366.31it/s]
